<a href="https://colab.research.google.com/github/superspray/TA/blob/main/NASA2024_CH2_V01_240714.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install konlpy
# !pip install selenium
!pip install scikit-multilearn --upgrade
!pip install transformers
!pip install accelerate -U
!apt-get update
# !apt install chromium-chromedriver
# !pip install googletrans==4.0.0-rc1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import os
import math
import random
import pandas as pd
import regex as re
import numpy as np
from typing import Optional, Sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, f1_score

import tensorflow as tf
import torch
import os
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split

from transformers import BertTokenizer
from transformers import ElectraForSequenceClassification,BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import time
import pandas as pd
import os
import random
import pickle
import re
from konlpy.tag import Okt
import math

from tqdm import tqdm
import torch
from torch import nn
from torch import Tensor
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, EarlyStoppingCallback, AutoModel, AutoConfig
from transformers import AutoTokenizer, TFAutoModel, AdamWeightDecay, AutoModelForSequenceClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping


# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import time
# import googletrans
# from googletrans import Translator
import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import tqdm

In [4]:
def seed_everything(seed: int = 2022):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

# load data


In [6]:
df_cat = pd.read_csv("/content/drive/MyDrive/NASA_2024/CLG2/DATA/24NASAC2_menu_cat.dat",   sep='|')
df_dev = pd.read_csv("/content/drive/MyDrive/NASA_2024/CLG2/DATA/24NASAC2_menu_dev.dat",   sep='|')
df_serial = pd.read_csv("/content/drive/MyDrive/NASA_2024/CLG2/DATA/24NASAC2_menu_serial.dat",   sep='|')
df_submit = pd.read_csv("/content/drive/MyDrive/NASA_2024/CLG2/DATA/24NASAC2_menu_submit.dat",   sep='|')
df_store = pd.read_csv("/content/drive/MyDrive/NASA_2024/CLG2/DATA/24NASAC2_store.dat",   sep='|')


print(df_cat.shape)
print(df_cat.head())
print(df_dev.shape)
print(df_dev.head())
print(df_serial.shape)
print(df_serial.head())
print(df_submit.shape)
print(df_submit.head())
print(df_store.shape)
print(df_store.head())

(746, 4)
  menu_cat1_cd menu_cat1 menu_cat2_cd menu_cat2
0            A   과일/샐러드류         A001  일반/야채샐러드
1            A   과일/샐러드류         A002     과일샐러드
2            A   과일/샐러드류         A003     과일통조림
3            A   과일/샐러드류         A004      과일화채
4            A   과일/샐러드류         A005   돼지고기샐러드
(8455628, 11)
   store_serial source  menu1_serial  menu1_avgamt  menu2_serial  \
0         86246     매장      580793.0          6500           NaN   
1         86246     매장      147488.0          7500           NaN   
2         86246     매장      900462.0          5500           NaN   
3         86246     매장     1939604.0          5500           NaN   
4         86246     매장       62536.0          5500           NaN   

   menu2_avgamt menu1_cat1_cd menu1_cat2_cd menu2_cat1_cd menu2_cat2_cd  \
0             0             O          O015           NaN           NaN   
1             0             Z          Z999           NaN           NaN   
2             0             O          O025           N

In [9]:
df_cat.columns

Index(['menu_cat1_cd', 'menu_cat1', 'menu_cat2_cd', 'menu_cat2'], dtype='object')

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Matplotlib 설정
import matplotlib.pyplot as plt

# 가정: 'category_column'이라는 이름의 열에서 value_counts()를 계산
value_counts = df_cat['menu_cat1'].value_counts()

# 결과를 바 차트로 표현

plt.rc('font', family='NanumBarunGothic')
# plt.figure(figsize=(10, 8))  # 그래프 크기 설정
# value_counts.plot(kind='bar')  # 바 차트로 표현
# plt.title('Category Value Counts')  # 그래프 제목
# plt.xlabel('Categories')  # x축 라벨
# plt.ylabel('Counts')  # y축 라벨
# plt.xticks(rotation=45)  # x축 라벨 회전 각도
# plt.show()
value_counts

menu_cat1
국/탕/찌개/전골류    120
빵/과자류          99
추가/사리          83
밥류             60
냉채/무침류         41
면류             36
구이류            33
음료류            31
주류             26
볶음류            26
튀김류            22
적/꼬치류          18
전/부침류          17
죽류             17
찜류             17
빙과류            17
만두류            15
조림류            14
회류             13
과일/샐러드류        13
데침/삶음류         10
떡류              9
장/장아찌류          5
묵류              2
김치류             1
기타              1
Name: count, dtype: int64

In [21]:
print("df_dev # unique store : ", df_dev.store_serial.nunique())
print("df_store # unique store : ", df_store.store_serial.nunique())
print("submit # unique store : ", df_submit.store_serial.nunique())
set(df_submit.store_serial) - set(df_dev.store_serial)

df_dev # unique store :  173622
df_store # unique store :  173636
submit # unique store :  66829


{7255,
 21718,
 39613,
 40398,
 54653,
 60179,
 92661,
 93389,
 106678,
 117993,
 123330,
 139228,
 160604,
 170469}

In [23]:
df_dev.is_answer.value_counts()

is_answer
0    8408556
1      47072
Name: count, dtype: int64

In [24]:
df_dev.menu1_cat1_cd.value_counts()

menu1_cat1_cd
Z    1909770
O    1612984
T     830954
N     470673
K     447411
B     435214
C     391362
W     369207
I     366096
Y     240908
L     158240
X     132974
F      90086
V      64936
R      61435
A      54826
H      52766
E      35626
M      31963
Q      29865
J      15029
S      14483
G      12227
U       8501
P       6448
D          1
Name: count, dtype: int64

In [25]:
df_dev[df_dev.is_answer == 1].menu1_cat1_cd.value_counts()

menu1_cat1_cd
T    9097
C    6073
O    5236
B    3889
K    3529
I    3016
W    2773
Y    2267
N    2239
F    1957
L    1236
X    1121
V     987
A     651
R     560
E     552
H     362
Q     298
M     247
U     229
S     225
J     189
G     126
Z     120
P      84
Name: count, dtype: int64

In [26]:
df_dev[df_dev.is_answer == 0].menu1_cat1_cd.value_counts()

menu1_cat1_cd
Z    1909650
O    1607748
T     821857
N     468434
K     443882
B     431325
C     385289
W     366434
I     363080
Y     238641
L     157004
X     131853
F      88129
V      63949
R      60875
A      54175
H      52404
E      35074
M      31716
Q      29567
J      14840
S      14258
G      12101
U       8272
P       6364
D          1
Name: count, dtype: int64

In [28]:
df_dev[df_dev.is_answer == 0]

,store_serial,source,menu1_serial,menu1_avgamt,menu2_serial,menu2_avgamt,menu1_cat1_cd,menu1_cat2_cd,menu2_cat1_cd,menu2_cat2_cd,is_answer
0,86246,매장,580793.0,6500,NaN,0,O,O015,NaN,NaN,0
1,86246,매장,147488.0,7500,NaN,0,Z,Z999,NaN,NaN,0
2,86246,매장,900462.0,5500,NaN,0,O,O025,NaN,NaN,0
3,86246,매장,1939604.0,5500,NaN,0,Z,Z999,NaN,NaN,0
4,86246,매장,62536.0,5500,NaN,0,O,O019,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
8455623,130780,매장,790711.0,99500,NaN,0,Z,Z999,NaN,NaN,0
8455624,130780,매장,2013543.0,79500,NaN,0,Z,Z999,NaN,NaN,0
8455625,130780,매장,2027252.0,20500,NaN,0,Z,Z999,NaN,NaN,0
8455626,130780,매장,842512.0,75500,NaN,0,T,T015,NaN,NaN,0
